In [9]:

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [15]:
random_phase = np.random.rand(224, 224) * 2 * np.pi

def create_speckle(phase):
    P = np.zeros(phase.shape)
    N = phase.shape[0]
    x = np.arange(-N//2, N//2)
    y = x
    X, Y = np.meshgrid(x, y)
    P[X**2 + (Y-10)**2 < 100] = 1

    D = np.exp(1j * random_phase)
    
    lam = 0.532e-6
    f1 = 0.2
    f2 = 0.1
    M = np.exp(1j * phase)
    H = D * P
    F = np.fft.fftshift(np.fft.fft2(M)) * H
    U = np.fft.ifft2(np.fft.ifftshift(F))
    U = U / (lam**2 * f1 * f2) 
    U = np.fliplr(U)
    I = np.abs(U)**2
    return I

In [17]:
import os
from tqdm import tqdm

path = 'dataset/train/phase'
vmax = 4.36167865194187e+28
for file in tqdm(os.listdir(path)):
    phase = Image.open(os.path.join(path, file)).convert('L')
    phase = np.array(phase)
    phase = phase / 255 * 2 * np.pi
    phase = cv2.resize(phase, dsize=(224, 224))
    speckle = create_speckle(phase)
    speckle = speckle / vmax * 225
    Image.fromarray(speckle.astype('uint8')).save(f'dataset/train/speckle/{file}')

100%|██████████| 700/700 [00:03<00:00, 175.81it/s]


In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os
from torch.utils.data import Dataset, DataLoader
from pytorch_msssim import ssim, ms_ssim # <--- 이 부분을 추가하세요
import torch.nn.functional as F

# --- 기본 설정 (이전과 동일) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
N = 224
LEARNING_RATE_MODEL = 1e-4
LEARNING_RATE_PHASE = 1e-3

import torch, math
import torch.nn as nn

# ───────────────── ① 고정 pupil 생성 ─────────────────
def make_pupil(N, radius_px=10, y_shift_px=-10):
    x = torch.arange(-N//2, N//2)
    X, Y = torch.meshgrid(x, x, indexing='ij')
    P = (((X+y_shift_px)**2 + Y**2) < radius_px**2).float()
    return P          # (N,N) real, 0/1

class SpeckleSimulator(nn.Module):
    def __init__(self, N=224, lam=0.532e-6, f1=0.2, f2=0.1,
                 radius_px=10, y_shift_px=-10):
        super().__init__()
        
        self.lam = lam
        self.f1 = f1
        self.f2 = f2
        self.radius_px = radius_px
        self.y_shift_px = y_shift_px
        
        diffuser_phase = torch.rand(N, N) * 2 * torch.pi
        self.diffuser = nn.Parameter(diffuser_phase, requires_grad=True)

        pupil_mask = make_pupil(N, radius_px, y_shift_px)
        self.register_buffer('P', pupil_mask)
        
    def forward(self, phase):
        
        D = torch.exp(1j * self.diffuser)
        H = D * self.P
        
        M = torch.exp(1j * phase)
        F = torch.fft.fftshift(torch.fft.fft2(M)) * H.cuda()
        U = torch.fft.ifft2(torch.fft.ifftshift(F))
        I = torch.abs(U)**2
        I = I / torch.max(I).item()
        I = torch.flip(I, dims=[-1])
        return I

# --- 🌟 데이터셋 클래스 정의 🌟 ---
class SpeckleDataset(Dataset):
    def __init__(self, phase_dir, speckle_dir):
        # 이미지 파일 경로 리스트 가져오기
        self.phase_paths = sorted(os.listdir(phase_dir))
        self.speckle_paths = sorted(os.listdir(speckle_dir))

        self.phase_list = []
        self.speckle_list = []

        # 각 이미지에 대한 위상을 저장할 딕셔너리
        for path in self.phase_paths:
            phase = Image.open(os.path.join(phase_dir, path)).convert('L')
            phase = np.array(phase)
            phase = phase / 255 * 2 * np.pi
            phase = cv2.resize(phase, dsize=(224, 224))
            self.phase_list.append(phase)
        
        for path in self.speckle_paths:
            speckle = Image.open(os.path.join(speckle_dir, path)).convert('L')
            speckle = np.array(speckle)
            speckle = speckle / 255
            speckle = cv2.resize(speckle, dsize=(224, 224))
            self.speckle_list.append(speckle)
            
    def __len__(self):
        return len(self.phase_list)
    
    def __getitem__(self, idx):
        phase = self.phase_list[idx]
        speckle = self.speckle_list[idx]
        phase = torch.from_numpy(phase)
        speckle = torch.from_numpy(speckle)
        return phase, speckle
    
train_dataset = SpeckleDataset('dataset/train/phase', 'dataset/train/speckle')
test_dataset = SpeckleDataset('dataset/test/phase', 'dataset/test/speckle')
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

model = SpeckleSimulator()
model = model.cuda()

loss_fn = torch.nn.L1Loss()
s = torch.tensor(1.0, requires_grad=True)
optimizer = torch.optim.Adam(list(model.parameters()) + [s], lr=5e-4)

Using device: cuda


In [24]:
optimizer = torch.optim.Adam(list(model.parameters()) + [s], lr=1e-4)

In [32]:
num_epoch = 100
for i in range(num_epoch):
    print(f'========= Epoch : {i} =========')
    model = model.train()
    train_loss = []
    for phase, speckle in train_loader:
        optimizer.zero_grad()
        phase = phase.type(torch.float32).cuda()
        speckle = speckle.type(torch.float32).cuda()
        
        output = model(phase.unsqueeze(1))
        output = output.squeeze(1)
        
        loss = loss_fn(s * output, speckle)
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())
    
    print(f'Epoch {i}, Train Loss {np.mean(np.array(train_loss)):.6f}')
    
    model = model.eval()
    test_loss = []
    with torch.no_grad():
        for phase, speckle in test_loader:
            phase = phase.type(torch.float32).cuda()
            speckle = speckle.type(torch.float32).cuda()
        
            output = model(phase.unsqueeze(1))
            output = output.squeeze(1)
            
            loss = loss_fn(s * output, speckle)
            test_loss.append(loss.item())
        
        print(f'Epoch {i}, Test Loss {np.mean(np.array(test_loss)):.6f}')
            

========= Epoch : 0 =========
Epoch 0, Train Loss 0.038105
Epoch 0, Test Loss 0.103604
========= Epoch : 1 =========
Epoch 1, Train Loss 0.037304
Epoch 1, Test Loss 0.103658
========= Epoch : 2 =========
Epoch 2, Train Loss 0.036178
Epoch 2, Test Loss 0.104012
========= Epoch : 3 =========
Epoch 3, Train Loss 0.035789
Epoch 3, Test Loss 0.104436
========= Epoch : 4 =========
Epoch 4, Train Loss 0.035296
Epoch 4, Test Loss 0.104661
========= Epoch : 5 =========
Epoch 5, Train Loss 0.034650
Epoch 5, Test Loss 0.104876
========= Epoch : 6 =========
Epoch 6, Train Loss 0.033301
Epoch 6, Test Loss 0.105140
========= Epoch : 7 =========
Epoch 7, Train Loss 0.033332
Epoch 7, Test Loss 0.105469
========= Epoch : 8 =========
Epoch 8, Train Loss 0.032797
Epoch 8, Test Loss 0.105585
========= Epoch : 9 =========
Epoch 9, Train Loss 0.032217
Epoch 9, Test Loss 0.105849
========= Epoch : 10 =========
Epoch 10, Train Loss 0.031664
Epoch 10, Test Loss 0.105628
========= Epoch : 11 =========
Epoch 11,

In [33]:
from PIL import Image
import numpy as np
import torch

image = Image.open('apple.png').convert('L')
image = np.array(image)
image = cv2.resize(image, dsize=(224,224))
phase = image / 255 * 2 * np.pi

phase = torch.from_numpy(phase).cuda()
phase = phase.type(torch.float32)
phase = phase.unsqueeze(0).unsqueeze(0)
output = model(phase)

output = output.squeeze(0).squeeze(0)
output = output.detach().cpu().numpy()
output = output / np.max(output) * 255
Image.fromarray(output.astype('uint8')).save('output.png')

In [21]:
image = Image.open('apple.png').convert('L')
image = np.array(image)
image = cv2.resize(image, dsize=(224,224))
phase = image / 255 * 2 * np.pi

speckle = create_speckle(phase)
speckle = speckle / np.max(speckle) * 225
Image.fromarray(speckle.astype('uint8')).save(f'speckle.png')